In [3]:
import datasus_dbc
from dbfread import DBF
import pandas as pd
import os
import glob


In [6]:
def converter_dbc_seguro(arquivo_dbc, arquivo_csv):
    """
    Converte DBC para CSV evitando erros de data
    """
    try:
        # Passo 1: Descomprimir DBC para DBF
        nome_dbf = arquivo_dbc.replace('.dbc', '.dbf')
        datasus_dbc.decompress(arquivo_dbc, nome_dbf)
        
        # Passo 2: Ler DBF com encoding correto
        table = DBF(nome_dbf, encoding='cp1252', raw=True)
        
        # Passo 3: Converter para DataFrame tratando campos problemáticos
        registros = []
        for record in table:
            registro_limpo = {}
            for campo, valor in record.items():
                if isinstance(valor, bytes):
                    # Decodificar bytes para string
                    valor_str = valor.decode('cp1252', errors='ignore').strip()
                    # Se o campo parece ser uma data (8 caracteres) mas está vazio
                    if len(valor_str) == 0 and len(valor) == 8:
                        registro_limpo[campo] = None
                    else:
                        registro_limpo[campo] = valor_str
                else:
                    registro_limpo[campo] = valor
            registros.append(registro_limpo)
        
        # Passo 4: Criar DataFrame e salvar
        df = pd.DataFrame(registros)
        df.to_csv(arquivo_csv, index=False, encoding='utf-8')
        
        print(f"✅ {arquivo_dbc} convertido com sucesso!")
        print(f"   Registros processados: {len(df)}")
        return len(df)
        
    except Exception as e:
        print(f"❌ Erro em {arquivo_dbc}: {str(e)}")
        return 0

# Teste com um arquivo primeiro
resultado = converter_dbc_seguro('TUBEBR24.dbc', 'tubebr24.csv')

if resultado > 0:
    print("\n🎉 FUNCIONOU!")
else:
    print("❌ Ainda há problemas.")

✅ TUBEBR24.dbc convertido com sucesso!
   Registros processados: 111521

🎉 FUNCIONOU!


In [6]:
# 1. Localizar todos os arquivos CSV
# Caminho da pasta desejada
pasta = '/mnt/c/Users/AcerGamer/Downloads/Tuberculose/arquivos_csv'  # Exemplo: '/home/seu_usuario/meus_csvs' ou 'C:/Users/SeuUsuario/Downloads/csvs'

# Monta o padrão para buscar todos os CSVs da pasta
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

print(f"Arquivos CSV encontrados: {len(arquivos_csv)}")

# 2. Carregar e concatenar
dfs = []
for arquivo in arquivos_csv:
    try:
        # Carregar com encoding universal
        df = pd.read_csv(arquivo, encoding='utf-8', low_memory=False)
        dfs.append(df)
        print(f"✓ {os.path.basename(arquivo)}: {len(df)} linhas")
    except UnicodeDecodeError:
        # Tentar latin1 se UTF-8 falhar
        df = pd.read_csv(arquivo, encoding='latin1', low_memory=False)
        dfs.append(df)
        print(f"✓ {os.path.basename(arquivo)} (latin1): {len(df)} linhas")
    except Exception as e:
        print(f"✗ Erro em {arquivo}: {str(e)}")

# 3. Concatenar se houver arquivos válidos
if dfs:
    dados_completos = pd.concat(dfs, ignore_index=True)
    
    # 4. Salvar arquivo final
    arquivo_final = "dados_tuberculose_completos.csv"
    dados_completos.to_csv(arquivo_final, index=False, encoding='utf-8')
    
    # 5. Validar resultado
    total_linhas = len(dados_completos)
    total_colunas = len(dados_completos.columns)
    print("\n✅ Concatenação concluída com sucesso!")
    print(f"Arquivo final: {arquivo_final}")
    print(f"Total de registros: {total_linhas:,}")
    print(f"Total de colunas: {total_colunas}")
    
    # 6. Amostra dos dados
    print("\nPrimeiras linhas do dataset:")
    print(dados_completos.head(2))
else:
    print("❌ Nenhum arquivo CSV válido para concatenar")

Arquivos CSV encontrados: 5
✓ tubebr20.csv: 86236 linhas
✓ tubebr21.csv: 91385 linhas
✓ tubebr22.csv: 103322 linhas
✓ tubebr23.csv: 109807 linhas
✓ tubebr24.csv: 111521 linhas

✅ Concatenação concluída com sucesso!
Arquivo final: dados_tuberculose_completos.csv
Total de registros: 502,271
Total de colunas: 97

Primeiras linhas do dataset:
   TP_NOT ID_AGRAVO  DT_NOTIFIC  NU_ANO  SG_UF_NOT  ID_MUNICIP  ID_REGIONA  \
0       2      A169    20200930    2020         15      150650       148.0   
1       2      A169    20200710    2020         15      150530       149.0   

    DT_DIAG  ANO_NASC  NU_IDADE_N  ... BENEF_GOV  AGRAVDROGA  AGRAVTABAC  \
0  20200930    2009.0      4011.0  ...       9.0         2.0         2.0   
1  20200707    1973.0      4046.0  ...       2.0         2.0         2.0   

   TEST_MOLEC  TEST_SENSI  ANT_RETRO  BAC_APOS_6  TRANSF  UF_TRANSF  \
0         5.0         NaN        NaN         NaN     2.0         15   
1         5.0         NaN        NaN           3     

In [8]:
df = pd.read_csv('/mnt/c/Users/AcerGamer/Downloads/Tuberculose/arquivos_csv/dados_tuberculose_completos.csv')
print(df.shape)
print(df.head())


/tmp/ipykernel_407/4136504825.py:1: DtypeWarning: Columns (29,30,34,36,37,43,47,49,53,54,56,57,58,59,60,61,63,64,69,87,91,92,93,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/mnt/c/Users/AcerGamer/Downloads/Tuberculose/arquivos_csv/dados_tuberculose_completos.csv')


(502271, 97)
   TP_NOT ID_AGRAVO  DT_NOTIFIC  NU_ANO  SG_UF_NOT  ID_MUNICIP  ID_REGIONA  \
0       2      A169    20200930    2020         15      150650       148.0   
1       2      A169    20200710    2020         15      150530       149.0   
2       2      A169    20201203    2020         22      220770       186.0   
3       2      A169    20200210    2020         24      240920       141.0   
4       2      A169    20210114    2021         43      430210       160.0   

    DT_DIAG  ANO_NASC  NU_IDADE_N  ... BENEF_GOV  AGRAVDROGA  AGRAVTABAC  \
0  20200930    2009.0      4011.0  ...       9.0         2.0         2.0   
1  20200707    1973.0      4046.0  ...       2.0         2.0         2.0   
2  20201119    1955.0      4065.0  ...       1.0         2.0         2.0   
3  20200208    1960.0      4059.0  ...       2.0         2.0         1.0   
4  20201226    1984.0      4035.0  ...       9.0         2.0         1.0   

   TEST_MOLEC  TEST_SENSI  ANT_RETRO  BAC_APOS_6  TRANSF  UF_